Lambda School Data Science

*Unit 2, Sprint 3, Module 2*

---


# Wrangle ML datasets 🍌


In today's lesson, we’ll work with a dataset of [3 Million Instacart Orders, Open Sourced](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2)!



### Setup

In [1]:
# Download data
import requests

def download(url):
    filename = url.split('/')[-1]
    print(f'Downloading {url}')
    r = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(r.content)
    print(f'Downloaded {filename}')

download('https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz')

Downloaded instacart_online_grocery_shopping_2017_05_01.tar.gz


In [2]:
# Uncompress data
import tarfile
tarfile.open('instacart_online_grocery_shopping_2017_05_01.tar.gz').extractall()

In [3]:
# Change directory to where the data was uncompressed
%cd instacart_2017_05_01

/Users/huanqingxu/Desktop/repos/DS-Unit-2-Applied-Modeling/module2/instacart_2017_05_01


In [4]:
# Print the csv filenames
from glob import glob
for filename in glob('*.csv'):
    print(filename)

products.csv
orders.csv
order_products__train.csv
departments.csv
aisles.csv
order_products__prior.csv


### For each csv file, look at its shape & head 

In [5]:
import pandas as pd
def preview():
    for filename in glob('*.csv'):
        df=pd.read_csv(filename)
        print(filename,df.shape)
        display(df.head())
        print('\n')
preview()

products.csv (49688, 4)


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13




orders.csv (3421083, 7)


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0




order_products__train.csv (1384617, 4)


...

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol




aisles.csv (134, 2)


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation




order_products__prior.csv (32434489, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


## The original task was complex ...

[The Kaggle competition said,](https://www.kaggle.com/c/instacart-market-basket-analysis/data):

> The dataset for this competition is a relational set of files describing customers' orders over time. The goal of the competition is to predict which products will be in a user's next order.

> orders.csv: This file tells to which set (prior, train, test) an order belongs. You are predicting reordered items only for the test set orders.

Each row in the submission is an order_id from the test set, followed by product_id(s) predicted to be reordered.

> sample_submission.csv: 
```
order_id,products
17,39276 29259
34,39276 29259
137,39276 29259
182,39276 29259
257,39276 29259
```

## ... but we can simplify!

Simplify the question, from "Which products will be reordered?" (Multi-class, [multi-label](https://en.wikipedia.org/wiki/Multi-label_classification) classification) to **"Will customers reorder this one product?"** (Binary classification)

Which product? How about **the most frequently ordered product?**

# Questions:

- What is the most frequently ordered product?
- How often is this product included in a customer's next order?
- Which customers have ordered this product before?
- How can we get a subset of data, just for these customers?
- What features can we engineer? We want to predict, will these customers reorder this product on their next order?

## What was the most frequently ordered product?

In [7]:
prior=pd.read_csv('order_products__prior.csv')

In [8]:
prior['product_id'].mode()

0    24852
dtype: int64

In [9]:
train=pd.read_csv('order_products__prior.csv')

In [11]:
product=pd.read_csv('products.csv')
product[product['product_id']==24852]

,product_id,product_name,aisle_id,department_id
24851,24852,Banana,24,4


## How often is this product included in a customer's next order?

There are [three sets of data](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b):

> "prior": orders prior to that users most recent order (3.2m orders)  
"train": training data supplied to participants (131k orders)  
"test": test data reserved for machine learning competitions (75k orders)

Customers' next orders are in the "train" and "test" sets. (The "prior" set has the orders prior to the most recent orders.)

We can't use the "test" set here, because we don't have its labels (only Kaggle & Instacart have them), so we don't know what products were bought in the "test" set orders.

So, we'll use the "train" set. It currently has one row per product_id and multiple rows per order_id.

But we don't want that. Instead we want one row per order_id, with a binary column: "Did the order include the product?"

Let's wrangle!

In [13]:
df=train.head(16).copy()
df['bananas']=df['product_id']==24852
df.groupby('order_id')['bananas'].any()

order_id
2    False
3    False
Name: bananas, dtype: bool

In [16]:
train['bananas'] = train['product_id']==24852
train_wrangled=train.groupby('order_id')['bananas'].any().reset_index()

In [17]:
target = 'bananas'
train_wrangled[target].value_counts(normalize=True)

False    0.853007
True     0.146993
Name: bananas, dtype: float64

In [ ]:
#Technique #2

In [ ]:
# Group by order_id, get a list of product_ids for that order
df.groupby('order_id')['product_id'].apply(list)

In [ ]:
def includes_bananas(product_ids):
    return 24852 in list(product_ids)


df.groupby('order_id')['product_id'].apply(includes_bananas)

In [ ]:
train = (train
         .groupby('order_id')
         .agg({'product_id': includes_bananas})
         .reset_index()
         .rename(columns={'product_id': 'bananas'}))

In [ ]:

train[target].value_counts(normalize=True)

## Which customers have ordered this product before?

- Customers are identified by `user_id`
- Products are identified by `product_id`

Do we have a table with both these id's? (If not, how can we combine this information?)

In [ ]:
BANANAS = 24852
prior[prior.product_id==BANANAS]

In [ ]:
banana_prior_order_ids = prior[prior.product_id==BANANAS].order_id


In [ ]:
# Look at the orders table, which orders included bananas?
orders = pd.read_csv('orders.csv')
orders.sample(n=5)

In [ ]:
# In the orders table, which orders included bananas?
orders[orders.order_id.isin(banana_prior_order_ids)]

In [ ]:
# Check this order id, confirm that yes it includes bananas
prior[prior.order_id==738281]

In [ ]:
banana_orders = orders[orders.order_id.isin(banana_prior_order_ids)]

In [ ]:
# In the orders table, which users have bought bananas?
banana_user_ids = banana_orders.user_id.unique()

## How can we get a subset of data, just for these customers?

We want *all* the orders from customers who have *ever* bought the product.

(And *none* of the orders from customers who have *never* bought the product.)

In [ ]:
# orders table, shape before getting subset
orders.shape

In [ ]:
# orders table, shape after getting subset
orders[orders.user_id.isin(banana_user_ids)]
orders.shape

In [ ]:
# IDs of *all* the orders from customers who have *ever* bought bananas
subset_order_ids = orders.order_id.unique()

In [ ]:
# order_products__prior table, shape before getting subset
prior.shape

In [ ]:
# order_products__prior table, shape after getting subset
prior = prior[prior.order_id.isin(subset_order_ids)]
prior.shape

In [ ]:
# order_products__train table, shape before getting subset
train.shape

In [ ]:
# order_products__train table, shape after getting subset
train = train[train.order_id.isin(subset_order_ids)]
train.shape

In [ ]:
# In this subset, how often were bananas reordered in the customer's most recent order?
train[target].value_counts(normalize=True)

## What features can we engineer? We want to predict, will these customers reorder this product on their next order?

In [ ]:
USER = 61911
prior = pd.merge(prior, orders[['order_id', 'user_id']])
prior['bananas'] = prior.product_id == BANANAS
# This user has ordered 196 products, 
df = prior[prior.user_id==USER]
df
# This person has ordered bananas six times
df['bananas'].sum()
# What percentage of orders?
df['bananas'].sum() / df['order_id'].nunique()